In [ ]:
import pandas as pd

In [ ]:
army_list1 = ['Alpha', 'Beta', 'S', 'Mike', 'Delta', 'Zulu', 'x-ray', 'yankee']
army_list2 = ['Alpha', 'Charlie', 'A', 'November','Delta']

i=0
while(i<max(len(army_list1[:]),len(army_list2[:]))):
    
    if(army_list1[i]==army_list2[i]):
        print('removing ', army_list1[i])
        army_list1.pop(i)
        army_list2.pop(i)
        
    elif(army_list1[i]=="S" and army_list2[i]=="A"):
        print('removing ', army_list1[i], 'from left')
        army_list1.pop(i)
        
    elif(army_list1[i]=="A" and army_list2[i]=="S"):
        print('removing ', army_list2[i], 'from left')
        army_list2.pop(i)
 
    
    i+=1

print(army_list1)
print(army_list2)

In [ ]:
army_list1 = ['Alpha', 'Beta', 'S', 'Mike', 'Delta']
army_list2 = ['Alpha', 'Charlie', 'A', 'November','Delta']

new_list1 = []
new_list2 = []

for left, right in zip(army_list1, army_list2):
    if left == "S" and  right == "A":
        new_list1.append(right)     
    elif left == "A" and right == "S":
        new_list2.append(left)
    elif not left == right:
        new_list1.append(left)
        new_list2.append(right)

print(new_list1)
print(new_list2)
        

In [ ]:
import bisect

age_borders = 2, 4, 13, 20, 65
age_descriptions = ('Baby', 'Toddler', 'Kid', 
                    'Teenager', 'Adult', 'Old Geezer')

age = float(input('And the age is ? '))

age_index = bisect.bisect_right(age_borders, age)
print(f'This is a {age_descriptions[age_index]}')

In [ ]:
def hello(func):
    def inner():
        print('Hello')
        return func()
    return inner

def goodbye(func):
    def inner():
        result = func()
        print('Goodbye')
        return result
    return inner

@hello
@goodbye
def a():
    print('I am "a"')

@hello
@goodbye
def b():
    print('I am "b"')

In [ ]:
a()
b()

In [ ]:
AFFIRMATION = frozenset({'y', 'yes', 'yeh', 'k', 'si', 'ok', 'yeah', 'yup'})
REJECTION = frozenset({'n', 'no', 'nope', 'nah', 'exit', 'quit', 'bye'})

def yesno(msg: str = "Yes or No? ") -> bool :
    while True:  # valid response loop
        response = input(msg)
        if response:
            if response.lower() in AFFIRMATION:
                return True
            elif response.lower() in REJECTION:
                return False
        print('Please answer Yes or No.')

In [ ]:
print(yesno('Testing: '))

In [ ]:
with open('/Users/stuartmoore/nums.txt') as nums:
    total = 0  # running total of valid inputs
    count = 0  # number of valid inputs
    for line_no, line in enumerate(nums, start=1):
        try:
            total += float(line.strip())
            count += 1
        except ValueError as e:
            print(f'line {line_no} has bad data: {line}')
if count > 0:
    print(f'Average is {total/count:.2f}')

# Game: rock paper scissors Spoke lizard
This generates the rules from a doc string. Very compact approach.

In [ ]:
#! /usr/bin/env python3                                                                                                
# rock paper scissors Spock lizard game, rules follow:
'''                                                                                                                    
scissors cuts paper                                                                                                    
paper covers rock                                                                                                      
rock crushes lizard                                                                                                    
lizard poisons Spock                                                                                                   
Spock smashes scissors                                                                                                 
scissors decapitates lizard                                                                                            
lizard eats paper                                                                                                      
paper disproves Spock                                                                                                  
Spock vaporizes rock                                                                                                   
rock crushes scissors                                                                                                  
'''

from random import choice
import readline

RULES = list(map(str.split, __doc__.lower().strip().split('\n')))

OPTIONS = ({winner for winner, verb, loser in RULES}
           | {loser for winner, verb, loser in RULES})

PROMPT = f"Make your choice from: {', '.join(sorted(OPTIONS))} \n " \
            f"(or press return alone to exit)\n" \
            f" choice: "
                                     
def check(playera, playerb, rules=RULES):
    for rule in rules:
        winner, verb, loser = rule
        if (playera, playerb) == (winner, loser):
            return playera, ' '.join(rule)
        if (playerb, playera) == (winner, loser):
            return playerb, ' '.join(rule)

print('\n\nWelcome to the game of rock paper scissors Spock lizard\n\nThe rules are:\n')
print(__doc__)
print()
while True:
    while True:
        player = input(PROMPT).lower()
        if not player or player in OPTIONS:
            break
    if not player:
        break
    computer = choice(list(OPTIONS))    
    try:
        winner, rule = check(player, computer)
        result = 'You WIN!' if player == winner else 'You Lose!'
    except TypeError as e:
        result, rule = "TIED", 'matched'
    print(f'{player} v. {computer} -> {result} \t{rule}')
    print()

# Understanding numbers written as words

In [ ]:
UNITS = [
        "zero", "one", "two", "three", "four", "five", "six", "seven", "eight",
        "nine", "ten", "eleven", "twelve", "thirteen", "fourteen", "fifteen",
        "sixteen", "seventeen", "eighteen", "nineteen",
        ]
TENS = ["", "", "twenty", "thirty", "forty", "fifty", "sixty", "seventy", "eighty", "ninety"]
SCALES = ["hundred", "thousand", "million", "billion", "trillion"]
number_words = {"and": (1, 0)}
for position, word in enumerate(UNITS):
    number_words[word] = (1, position)
for position, word in enumerate(TENS):
    number_words[word] = (1, position * 10)
for position, word in enumerate(SCALES):
    number_words[word] = (10 ** (position * 3 or 2), 0)

POSITION_WORDS = {'first':1, 'second':2, 'third':3, 'fifth':5, 'eighth':8, 'ninth':9, 'twelfth':12}
SUFFIXES = [('ieth', 'y'), ('th', '')]

def words_to_num(words):
    words = words.replace('-', ' ')
    words = words.replace(',', ' ')
    current = result = 0
    for word in words.strip().split():
        if word in POSITION_WORDS:
            scale, increment = 1, POSITION_WORDS[word]
        else:
            for ending, replacement in SUFFIXES:
                if word.endswith(ending):
                    word = f"{word[:-len(ending)]}{replacement}"
            if word not in number_words:
                raise ValueError(f"Illegal word: {word}")
            scale, increment = number_words[word]
        current = current * scale + increment
        if scale > 100:
            result += current
            current = 0
    return result + current

# Temperature conversions

In [ ]:
''' temperature conversion '''

def tempconv_F_to_C(temp):
    return (temp - 32) / 1.8

def tempconv_C_to_F(temp):
    return (temp * 1.8) + 32

def tempconv_Ré_to_C(temp):
    return temp * 5/4

def tempconv_C_to_Ré(temp):
    return temp * 4/5

def tempconv_Ré_to_F(temp):
    return tempconv_C_to_F(tempconv_Ré_to_C(temp))

def tempconv_F_to_Ré(temp):
    return tempconv_C_to_Ré(tempconv_F_to_C(temp))

# build dictionary of temperature conversion functions, 
# (dropping from the keys the prefix of tempconv_ and removing the _ before and after to)
prefix = 'tempconv_'
prefix_len = len(prefix)
tempconv_funcs = {name[prefix_len:].replace('_to_', ' to '): func for name, func in globals().items() 
                 if callable(func) and name.startswith(prefix)
                }
# define set of temperature units available
UNITS = set([unit for temp in [k.partition(' to ') 
                for k in tempconv_funcs.keys()] 
                for unit in temp if unit != ' to '])
ORIG = 'C'  # default original temp
NEW = 'F'  # default to convert to

def temp_convert(temp, orig, new):
    if not(isinstance(temp, int) or isinstance(temp, float)):
        raise ValueError('Valid temperature not provided')
    if not orig or not new:
        raise ValueError('Original and/or New units not specified')
    func = tempconv_funcs.get(f'{orig} to {new}')
    if not func:
        raise TypeError(f'No coversion available for {orig} to {new}')
    return func(temp)

print('Welcome to this temperature conversion programme\n')
print(f'The following units are accepted: {", ".join(UNITS)}')
print('and the following conversions:-', end='')
print('', *tempconv_funcs.keys(), sep='\n * ')

while True:  # do another conversion loop
    while True:  # get valid unit selections from user 
        orig = input(f'What units to convert from? [{ORIG}]  ')
        if not orig:
            orig = ORIG
        new = input(f'What units to convert to? [{NEW if orig != NEW else ORIG}] ')
        if not new:
            new = NEW if orig != NEW else ORIG
        if orig != new and orig in UNITS and new in UNITS:
            break
        print(f'You need to select from the following units: {", ".join(UNITS)}')

    while True:  # get a numeric input for temperature
        try:
            temp = float(input(f'Temperature ({orig}): '))
            break
        except:
            print('Numeric value required')

    try:
        print(f'{temp:.2f} degrees {orig} is {temp_convert(temp, orig, new):.2f} degrees {new}')
    except ValueError as e:
        print(e)
    except TypeError as e:
        print(e)

    if input('Another? [Yes/no] ').lower() not in ('y', 'yes', '1', 'yup', 'ok', ''):
        break

# validated numeric input (int or float)

In [ ]:
    def get_num(msg):
        'prompt user with msg and return valud numerical input, int or float'
        while True:    
            response = input('Some prompt: ')
            num = None
            try:
                num = int(response)
                break
            except ValueError as e:
                try:
                    num = float(response)
                    break
                except ValueError:
                    pass
            print('Please enter a number')
        return num

    num = get_num('Please enter a number: ')
    print(f"You entered: {num} of type {type(num)}")

# Crude way to check for a prime number

In [ ]:
def is_prime(num):
    if num == 1: return False
    if num in (2, 3): return True
    if not num % 2: return False
    for value in range(3, int(num**0.5+1), 2):
        if not num % value: return False
    return True

# Determing day of week for any date (no libraries)


![formulae](https://cs.uwaterloo.ca/~alopez-o/math-faq/img297.gif)

k is day (1 to 31)

m is month (1 = March, ..., 10 = December, 11 = Jan, 12 = Feb) Treat Jan & Feb as months of the preceding year

C is century (1987 has C = 19)

Y is year (1987 has Y = 87 except Y = 86 for Jan & Feb)

W is week day (0 = Sunday, ..., 6 = Saturday)

From: https://cs.uwaterloo.ca/~alopez-o/math-faq/node73.html

In [ ]:
''' determine day of week for any Gregorian date '''

# based on formulae at https://cs.uwaterloo.ca/~alopez-o/math-faq/node73.html

DOW = dict(enumerate(('Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday')))
MONTHS = dict(enumerate(('January', 'February', 'March', 'April',
                       'May', 'June', 'July', 'August',
                       'September', 'October', 'November', 'December'), start=1))



def is_leap_year(year):
    'return True/False if year is a leap year'
    return year % 4 == 0 and (year % 100 != 0 or year % 400 == 0)

def get_date(msg):
    'get date from user and return tuple of dd, mm, yyyy'
    while True:
        response = input(msg).strip().split('/')
        if response and len(response) == 3:
            try:
                dd, mm, yyyy = [int(x) for x in response]
                if not 1 <= dd <= 31:
                    raise IndexError('Day must be between 1 and 31')
                if not 1 <= mm <= 12:
                    raise IndexError('Month must be between 1 and 12')
                if not 1582 <= yyyy <= 9999:
                    raise IndexError('Year must be between 1582 and 9999')
                if mm == 2 and dd > 28:
                    if not is_leap_year(yyyy) or dd > 29:
                        raise IndexError(f'Too many days for {MONTHS[2]} in {yyyy}')
                if dd == 31 and mm not in (1, 3, 5, 7, 8, 10, 12):
                    raise IndexError(f'Too many days for {MONTHS[mm]}')
                return dd, mm, yyyy
            except ValueError as e:
                print(f'{response} can not be understood as a date such as 18/05/1958')
            except IndexError as e:
                print(e)
        else:
            print('A response in the required format is expected. Please try again.')

def day_of_week(dd, mm, yyyy):
    'return day of week for date dd/mm/yyyy'
    mm = mm - 2  # march = 1, jan = 11
    if mm < 1:
        mm + 12
    if mm >= 11:  # year - 1 if month is jan or feb
        yyyy -= 1

    C = int(yyyy / 100)
    Y = yyyy % (C * 100)
    W = (dd + int(2.6 * mm - 0.2) - 2 * C + Y + int(Y/4) + int(C/4)) % 7

    return DOW[W]

dd, mm, yyyy = get_date('Please enter date of birth in form: dd/mm/yyyy: ')
print(f'{dd}/{mm}/{yyyy} is a {day_of_week(dd, mm, yyyy)}')

# example of using itertools islice to chunk something up

In [ ]:
from itertools import islice

def outcomes():
    'check scores are in range, return list of results'

    def chunk(it, size):
        'split a sequence into tuple chunks of same size'
        it = iter(it)
        return iter(lambda: tuple(islice(it, size)), ())

    Final = []
    name = ['James', 'Rod', 'Kate', 'Ann', 'Nick']
    count = [1,3,6,-2,7,8,11,-2,10,4,3,1,5,-999,9]
    results = {name_: counts_ for name_, counts_ in zip(name, list(chunk(count, 3)))}
    print(results)

    for name_, counts in results.items():
        if all( -1 < count_ < 11 for count_ in counts ):
            Final.append(f'{name_} passed.\n')
        elif -999 in counts:
            Final.append(f'{name_} had an error.\n')
        else:
            Final.append(f'{name_} failed.\n')

    return Final

print(''.join(outcomes()))

# function to merge strings (dropping any overlapping text)

In [ ]:
def str_merge(left: str, right: str) -> str:
    'return merged string, without duplication in overlap'
    chars = 0  # number of overlapping characters
    for ind in range(1, min(len(right), len(left)) + 1):
        print(left[-ind:], right[:ind], left.endswith(right[:ind]))  # DEBUG line
        if left.endswith(right[:ind]):
            chars = ind
    return left + right[chars:]
        
# usage example:        
print('result: ', str_merge('abc', 'cfghiabc'))

# nest list sorting using itemgetter from operator

In [ ]:
from operator import itemgetter

data = [['name3', 0.3, 10], ['name1', 0.2453022035139001, 5], ['name2', 0.2116333563690437, 15]]
print(data)
data.sort(key=itemgetter(1))  # i.e. sort on second element 
print(data)

# using bisect to find a dictionary boundary entry

In [ ]:
import bisect

RATES = {0: 0.0005,
         100000: 0.0010, 
         200000: 0.0015, 
         250000: 0.0025, 
         500000: 0.0040, 
         1000000: 0.0055, 
         2000000: 0.0060, 
         5000000: 0.0070}

RATES_ = list(RATES.keys())

def findInterestEarned2(p, m, t):

    def calc_for_rate():
        if p in RATES_:
            rate_index = RATES_.index(p)
        else:
            rate_index = bisect.bisect(RATES_, p) - 1
        rate = RATES[RATES_[rate_index]]
        calc = (m * 12) + ((p + ( m * 12 )) * rate )
        interest = ( (p + (m * 12)) * rate )
        return calc, interest

    counter = 0
    interest = 0

    if p >= 0 and m >= 0 and t > counter:
        for counter in range(t):
            calc, interest_ = calc_for_rate()
            interest += interest_
            p += calc
    return (interest)

print(findInterestEarned2(200000, 12, 12))

In [ ]:
def file ():
    letter_groups = ["abc", "def", "ghi", "jkl", "mno", "pqrs", "tuv", "wxyz"]
    result=[]
    with open("first.txt","rt") as first:
        for line in first:
            for character in line:
                for idx, seq in enumerate(letter_groups, start = 2):
                    if character in seq:
                        result.append(idx)
                        break
        return "".join([str(idx) for idx in result])

In [ ]:
print(file())

In [ ]:
def file2 ():
    letter_groups = {"abc": 2, "def":3 , "ghi": 4, "jkl": 5, "mno": 6, "pqrs": 7, "tuv": 8, "wxyz": 9}
    result=[]
    with open("first.txt","rt") as first:
        for line in first:
            for character in line:
                seq = [seq for seq in letter_groups.keys() if character in seq]
                if seq:
                    result.append(letter_groups[seq[0]])
        return "".join([str(idx) for idx in result])

In [ ]:
print(file2())

In [ ]:
def indexofmin(list1):
    'return index position of lowest value in list (first occurence)'
    min_idx = 0  # assume first entry is lowest
    lowest = list1[0]  # using variable to save looking up list1[index] every loop
    idx = 1  # start comparisons from element 1
    for value in list1[1:]:
        if value < lowest:
            min_idx = idx
            lowest = value
        idx += 1
    return min_idx

x = input("Enter a list of space separated numbers: ")
try:
    list1 = [float(num) for num in x.split()]
except ValueError as e:
    print('invalid number in list')
else:
    result = indexofmin(list1)
    print(result)

In [ ]:
def chunks(array, size):
    'Yield successive sized chunks from array'
    for idx in range(0, len(array), size):
        yield array[idx:idx+size]

Array = [[1,2], [3, 4], [5, 6], [7, 8]]
new_array =[element for element in chunks(arr, 2)]
a = new_array[0]
b = new_array[1]

In [ ]:
d = [101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114]
c = [201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214]
h = [301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314]
s = [401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414]
deck = [d, c, h, s]

firstColor = int(input("Color of the first card: (1 = Diamonds, 2 = Clubs, 3 = Hearts, 4 = Spades)\n"))
firstValue = int(input("Value of the first card: (1 = A, 11 = J, 12 = Q, 13 = K)\n"))
secondColor = int(input("Color of the second card: (1 = Diamonds, 2 = Clubs, 3 = Hearts, 4 = Spades)\n"))
secondValue = int(input("Value of the second card: (1 = A, 11 = J, 12 = Q, 13 = K)\n"))

dP = []
cP = []
hP = []
sP = []
playerDeck = [dP, cP, hP, sP]

def firstCard(i):
    switcher = {
        1: dP.insert(0, 100 + firstValue),
        2: cP.insert(0, 200 + firstValue),
        3: hP.insert(0, 300 + firstValue),
        4: sP.insert(0, 400 + firstValue)
        }
    return switcher.get(i)

firstCard(firstColor)
firstCard(secondColor)

print(playerDeck)

In [ ]:
firstCard(4)

In [ ]:
dP

In [ ]:
entry = 'dummy'
scores = []
while entry:  # keep looping until entry is empty
    entry = input('Enter next score: ')
    if entry:  # if entry is not empty
        try:
            score = int(entry)  # convert string to an intger, if fails except will catch it
            scores.append(score)
        except ValueError as e:
            print('Sorry, do not understand that score')

total = sum(scores)
number_of_scores = len(scores)
average = total / number_of_scores

print(f'Total: {total}, average: {average:.2f} of {number_of_scores} scores')

In [ ]:
def move(my_history, their_history, my_score, their_score):
    ''' Arguments accepted: my_history, their_history are strings.
    my_score, their_score are ints.
    
    Make my move.
    Returns 'c' or 'b'. 
    '''

    # my_history: a string with one letter (c or b) per round that has been played with this opponent.
    # their_history: a string of the same length as history, possibly empty. 
    # The first round between these two players is my_history[0] and their_history[0].
    # The most recent round is my_history[-1] and their_history[-1].
    
    # Analyze my_history and their_history and/or my_score and their_score.
    # Decide whether to return 'c' or 'b'.
    print(f'moving - {my_history}, {their_history}, {my_score}, {their_score}')
    if len(my_history)==0:
        return 'c'
    if their_history[:-3]=='bbc': #If last 3 has 2 Betrays, Betray
        return 'b'
    if their_history[:-3]=='cbb':
        return 'b'
    if their_history[:-3]=='bcb':
        return 'b'
    if their_history[:-3]=='ccb': #If last 3 has 2 Colludes, collude
        return 'c'
    if their_history[:-3]=='cbc':
        return 'c'
    if their_history[:-3]=='bcc':
        return 'c'
    if their_history[:-3]=='bbb': #If last 3 has 3 Betrays, BETRRAY
        return 'b'
    if their_history[:-3]=='ccc': #If last 3 has 3 colludes, collude x3 >.<
        return 'c'

if True:
    
    #Test Move function
    print(move('', '', 0, 0))
    print(move('bcbcbc','bcbcbc',10,-10))
    print(move('bcbcbcbcbcbbcbbcbbbcbcbccbcbcccccc', 'bcbcbcbcbcbbcbbcbbbcbcbccbcbcccccc', -30, -10))
    print(move('ccccc', 'ccccc', 0, 0))
    print(move('bbbbb', 'bbbbb', 0, 0))
    print("All tests passed")

In [ ]:
    import time
    import random

    print(
    """American Military History Database, version 0.0.3.1
    Coded by Heath
    -
    Please note that this program is a work in progress and new articles, topics, and other
    info is being added every day.
    -
    -
    -
    Search any weapon, battle, vehicle, or person involved in any of the United
    State's past or present involvement in any amred conflicts below.
    -
    """)

    # primary dictionary with prefered capitalisation
    WW2_keywords = {
        "WW2": ["WW2", "ww2", "World War 2", "world war 2", "World War Two", "world war two"],
        "WW2 Keywords": ["World War 2 Keywords", "world war 2 keywords", "World War 2 keywords", "World War Two Keywords", "world war two keywords", "World War Two keywords", "WW2 Keywords", "ww2 keywords", "WW2 keywords", "ww2 Keywords"],
        "WW2 Weapons": ["WW2 weapons", "ww2 weapons", "WW2 Weapons"],
        "WW2 Vehicles": ["WW2 vehciles", "ww2 vehicles", "WW2 Vehicles"],
        "WW2 Battles": ["WW2 battles", "ww2 battles", "WW2 Battles"],
        "WW2 Fronts": ["WW2 fronts", "ww2 fronts", "WW2 Fronts"],
        "WW2 Theatres": ["WW2 theatres", "ww2 theatres", "WW2 Theatres"],
        "WW2 Axis Powers": ["Axis powers", "Axis Powers", "the Axis Powers", "The Axis Powers", "The axis powers", "the axis powers"],
        "WW2 Allied Powers": ["Allied powers", "Allied Powers", "the Allied Powers", "The Allied Powers", "The allied powers", "the allied powers"],
        "WW2 Major Events ": ["WW2 major events", "WW2 Major Events", "ww2 major events", "WW2 Major events", "ww2 Major Events"],
        "WW2 Leaders": ["WW2 leaders","ww2 leaders", "WW2 Leaders", "ww2 Leaders"],
        "WW2 Officers": ["WW2 famous officers", "ww2 famous officers", "ww2 Famous officers"]
        }

    # copy of above with everything lowercased, and the original key included for lookup on primary
    ww2_keywords_lower = {key.lower(): (key, [words.lower() for words in terms]) 
                          for key, terms in WW2_keywords.items()}

    while True:  # continue inviting user to search until they enter return on its own
        s = input("Search: (blank to exit) ").lower()
        if not s:  # no input so user does not want to search anymore
            break

        if s in ww2_keywords_lower['ww2'][1]:
            print("""(paragraph) For more information on World War Two, search 'WW2 Keywords'""")

        elif s in ww2_keywords_lower['ww2 keywords'][1]:
            print("The following are keywords regarding 'World War Two':")
            print(*WW2_keywords['WW2 Keywords'], sep='\n')    

        else:
            if s in ww2_keywords_lower:
                original_key = ww2_keywords_lower[s][0]  # find original primary key
                print(f"The following articles have been found on {original_key}: ")
                print(*WW2_keywords[original_key], sep='\n')          
            else:
                print("ERROR; Article does not exist, be more specific.")

In [ ]:
from random import randint

def dice_roll():
    return randint(1,12)

def getpot(prompt):
    while True:
        response = input(prompt)
        if not response:
            return None
        try:
            cash = int(response)
            if cash < 1:
                raise ValueError("Please enter a positive number for the bet")
            break
        except ValueError as e:
            print(e)

    return cash


print("welcome to the casino, how about a game of lucky sevens?\n")
print("the goal of the game is to roll the dice and roll a total of 7\n")                    

while True:
    pot = getpot("Please enter the amount of money you want to bet (return to exit)\n")
    if not pot:
        break

    maxpot = pot
    total_rolls = 0
    table = []

    while pot > 0 and total_rolls < 1000:
        print(f'Pot: ${pot}')
        dice_roll_total = dice_roll()
        win = dice_roll_total == 720
        table.append((win, pot))
        pot += 4 if win else -1
        if pot > maxpot:
            maxpot = pot
        total_rolls += 1
        print(f"{'Win' if win else 'Lose'} -> roll #{total_rolls:4}: {dice_roll_total:2}   ", end="")

    print(f'Final pot {pot}')
    print(f'Highest pot {maxpot}')
    print(f'Number of rolls {total_rolls}')
    print()

    # table
    print('Status | Roll #  | Pot')
    print('========================')
    for idx, (win, pot) in enumerate(table):
        print(f"{'Win ' if win else 'Lose'}     {idx + 1:4}     {pot:4}   ")

In [41]:
list1=['a1','b1','c1']  
list2 = ['a2','b2', 'c2']

data1= 'c1'
data2= 'c2'
for d1, d2 in zip(list1, list2):
    if data1 == d1 and data2 == d2:
        print('yes')
        break
else:
    print('no')

yes


In [92]:
    trigger = 10  # each <trigger> words, insert <marker> before the word
    marker = '^'  # test to insert before the <trigger># word
    words_count = -1
    with open('random.txt') as words_file, open('new_randon.txt', 'w') as new_file:
        words_count = 0
        for line in words_file:
            words = line.split()
            words_num = len(words)
            new_line = ""
            for idx, word in enumerate(words, start=1):
                words_count += 1
                if words_count == trigger:
                    new_line += marker
                    words_count = 0
                new_line += word + (' ' if idx < words_num else '')
            new_file.write(new_line + '\n')    